<a href="https://colab.research.google.com/github/adi0229/ML-DL/blob/master/keras_lstm_generate_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# language model by keras

import keras

Using TensorFlow backend.


In [2]:
keras.__version__

'2.2.4'

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
import os

os.getcwd()

'/content'

更换语料库美剧《老友记》剧本

In [5]:
import numpy as np

text = open('/content/gdrive/My Drive/friends.txt').read().lower()
print("Coprus length:", len(text))

Coprus length: 4870645


In [6]:
type(text)

str

In [0]:
text = text[:300000]

In [8]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0,len(text)- maxlen, step):
  sentences.append(text[i:i+ maxlen])
  next_chars.append(text[i+ maxlen])

print('Number of sequences:',len(sentences))

Number of sequences: 99980


In [9]:
sentences[:10]

['101 the one where monica gets a new roommate (the pilot-the ',
 ' the one where monica gets a new roommate (the pilot-the unc',
 'e one where monica gets a new roommate (the pilot-the uncut ',
 'ne where monica gets a new roommate (the pilot-the uncut ver',
 'where monica gets a new roommate (the pilot-the uncut versio',
 're monica gets a new roommate (the pilot-the uncut version)\n',
 'monica gets a new roommate (the pilot-the uncut version)\n[sc',
 'ica gets a new roommate (the pilot-the uncut version)\n[scene',
 ' gets a new roommate (the pilot-the uncut version)\n[scene: c',
 'ts a new roommate (the pilot-the uncut version)\n[scene: cent']

In [10]:
next_chars[:10]

['u', 'u', 'v', 's', 'n', '[', 'e', ':', 'e', 'r']

In [0]:
chars = sorted(list(set(text)))

In [12]:
chars[:10] # unique chars in the corpus

['\n', ' ', '!', '"', '$', '&', "'", '(', ')', ',']

In [13]:
print("Unique characters:", len(chars))

Unique characters: 57


In [0]:
char_indices = dict((char, chars.index(char)) for char in chars)

In [15]:
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '$': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '/': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '4': 17,
 '5': 18,
 '6': 19,
 '7': 20,
 '8': 21,
 '9': 22,
 ':': 23,
 ';': 24,
 '?': 25,
 '[': 26,
 ']': 27,
 'a': 28,
 'b': 29,
 'c': 30,
 'd': 31,
 'e': 32,
 'f': 33,
 'g': 34,
 'h': 35,
 'i': 36,
 'j': 37,
 'k': 38,
 'l': 39,
 'm': 40,
 'n': 41,
 'o': 42,
 'p': 43,
 'q': 44,
 'r': 45,
 's': 46,
 't': 47,
 'u': 48,
 'v': 49,
 'w': 50,
 'x': 51,
 'y': 52,
 'z': 53,
 '}': 54,
 '—': 55,
 '’': 56}

In [16]:
print('Vectorizion...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorizion...


生成文本 "温度（temperatures）" 实验……

## 构建长短时记忆（LSTM）神经网络

In [17]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
    

W0625 09:37:51.169493 140571770709888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0625 09:37:51.193822 140571770709888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0625 09:37:51.197820 140571770709888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [18]:
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.01),loss='categorical_crossentropy')

W0625 09:37:51.487029 140571770709888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0625 09:37:51.496897 140571770709888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
  

In [20]:
import random
import sys

for epoch in range(1, 20):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.1, 0.5, 0.8,]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 100 characters
        for i in range(100):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

W0625 09:37:51.658761 140571770709888 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


epoch 1


W0625 09:37:52.468288 140571770709888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1
99980/99980 [==============================] - 128s 1ms/step - loss: 2.0813
--- Generating with seed: "ever.
all: oh, come on! come on!
monica: no. not after what "
------ temperature: 0.1
ever.
all: oh, come on! come on!
monica: no. not after what i wants the can the can the can the cand the can the cane the can the can the can she cane the can t
------ temperature: 0.5
 the can the cane the can the can the can she cane the can the ceaing i see oking get are happen she caille the paren the to leas it said on the could the thing
------ temperature: 0.8
 caille the paren the to leas it said on the could the thinge bo pits ick there fino me, nok, some cand wint?
monica: monica-eblige tome this men to wealing the
epoch 2
Epoch 1/1
99980/99980 [==============================] - 129s 1ms/step - loss: 1.6337
--- Generating with seed: "aves her head? she said that if you want to break the bad bo"
------ temperature: 0.1
aves her head? she said that if you want to break the bad book 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


s like her. (hand
------ temperature: 0.8
are there.]
monica: (stips on the termore is like her. (hands the others) what about you as and me, and he is?
phoebe: you're that every!
(kirsy back to the too
epoch 13
Epoch 1/1
99980/99980 [==============================] - 132s 1ms/step - loss: 1.1984
--- Generating with seed: "his pop-tart?
ross: hey, i might!
phoebe: sorry. ..y'know, t"
------ temperature: 0.1
his pop-tart?
ross: hey, i might!
phoebe: sorry. ..y'know, the line and susan any mone with the restaurant, you know what i mean, i mean, i don't know what i do
------ temperature: 0.5
aurant, you know what i mean, i mean, i don't know what i don't know it. the has the too. i think i mean, and i was the bading to say here?
ross: no. so you got
------ temperature: 0.8
mean, and i was the bading to say here?
ross: no. so you got to hold it.
ross: how's i could be me.
joey: you know. (she doesn't look on his let your guy her re
epoch 14
Epoch 1/1
99980/99980 [==========================

In [22]:
for epoch in range(1, 3):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=30)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.1, 0.5, 0.8,]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 100 characters
        for i in range(100):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/30
99980/99980 [==============================] - 130s 1ms/step - loss: 0.9971
Epoch 2/30
99980/99980 [==============================] - 130s 1ms/step - loss: 0.9915
Epoch 3/30
99980/99980 [==============================] - 130s 1ms/step - loss: 0.9875
Epoch 4/30
99980/99980 [==============================] - 131s 1ms/step - loss: 0.9838
Epoch 5/30
99980/99980 [==============================] - 129s 1ms/step - loss: 0.9829
Epoch 6/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9836
Epoch 7/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9783
Epoch 8/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9769
Epoch 9/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9730
Epoch 10/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9751
Epoch 11/30
99980/99980 [==============================] - 132s 1ms/step - loss: 0.9743
Epoch 12/30
99980/99980 [========

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


ure rachel) sorryaticalice.
chandler: i'm sorry, bint.
chandler: oh.
janicay, list
epoch 2
Epoch 1/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9442
Epoch 2/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9412
Epoch 3/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9353
Epoch 4/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9377
Epoch 5/30
99980/99980 [==============================] - 129s 1ms/step - loss: 0.9373
Epoch 6/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9375
Epoch 7/30
99980/99980 [==============================] - 129s 1ms/step - loss: 0.9332
Epoch 8/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9317
Epoch 9/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9336
Epoch 10/30
99980/99980 [==============================] - 128s 1ms/step - loss: 0.9319
Epoch 11/30
99980/99980 [=============

to do:

English lyrics as corpus